In [454]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [455]:
df = pd.read_csv('film_festival_new.csv')

In [456]:
df = df.astype({'prodYear': 'Int64', 'runtime': 'Int64'})

# 점수 알고리즘
### 서울독립영화제, 수상작, (서울독립영화제)중복 출품, (서울독립영화제)중복 수상을 기준으로 점수를 부여
- 서울독립영화제가 전체 작품 3362개 중에서 1073개를 차지하고, 두 번째부터는 315개의 작품으로 1위와의 작품 개수의 차이가 많다.

- 서울독립영화제는 독립영화계에서 권위 있는 영화제이므로 다른 영화제 출품작들과의 차이를 둔다.

- 서울독립영화제의 수상작 중에서 '최우수상', '대상', '관객상'은 심사위원, 관객으로부터 높은 인정을 받은 작품으로 다른 수상작과의 차이를 둔다.

- 기타 영화제의 수상작은 서울영화제에서 수상하지 못 한 작품보다 작품성이 높다고 가정했다.

### 점수 기준
- 서울독립영화제 출품작들에 대해 5점
- 서울독립영화제에서 '관객상','대상','최우수' 탄 작품들은 15점 추가, 이외의 상에 대해서는 10점 추가
- 다른 영화제의 수상작에 대해서는 10점
- 서울독립영화제에 중복 출품한 감독의 작품에 대해서는 5점 추가
- 서울독립영화제에서 중복 수상한 감독에 대해서는 5점 추가 (총 3명의 감독)


In [457]:
df['score'] = 0
df = df.astype({'score': 'Int64'})
df_seoul = df.copy()
df[df['film_festival'] == '서울독립영화제']['award'].unique()
#서울 독립영화제 상 종류

array([nan, '새로운선택상', '새로운시선상', '특별언급', '관객상', '최우수장편상', '독불장군상', '독립스타상',
       '최우수단편상', '집행위원회 특별상', '열혈스태프상', '심사위원상', '대상', '새로운 선택상',
       '심사위원 특별언급', '최우수작품상', '우수작품상', '새로운선택 심사위원 특별언급', '독립스타상-배우부문',
       '심사위원특별상', '심사위원특별언급', '열혈스태프상 (김현아)', '특별상', 'KT&G 상상마당상',
       '최우수 작품상', '우수 작품상'], dtype=object)

In [458]:
df_seoul.loc[(df_seoul.film_festival == '서울독립영화제'),'score'] = 5
list_0 = np.array(df_seoul['score'])
len(list_0)

3362

### 서울독립영화제에서 1073개의 작품 중 39개의 작품이 '대상', '최우수상', '관객상'을 탔다.
- 전체 수상작 데이터 수는 381개 (서울독립영화제가 아닌 영화제에서 수상한 작품이 265개)
- 서울독립영화제에서 수상작은 116개 (대상,최우수상,관객상을 제외한 상을 탄 작품이 77개)

In [459]:
df_seoul_award_big_prize = df.copy()
df_seoul_award_big_prize.loc[((df_seoul_award_big_prize.film_festival == '서울독립영화제') 
                       & (df_seoul_award_big_prize['award'].str.contains('대상'))),'score'] = 5
df_seoul_award_big_prize.loc[((df_seoul_award_big_prize.film_festival == '서울독립영화제') 
                        & (df_seoul_award_big_prize['award'].str.contains('관객'))),'score'] = 5
df_seoul_award_big_prize.loc[((df_seoul_award_big_prize.film_festival == '서울독립영화제') 
                        & (df_seoul_award_big_prize['award'].str.contains('최우수'))),'score'] = 5

list_1 = np.array(df_seoul_award_big_prize['score'])
print(len(list_1))
df_seoul_award_big_prize[df_seoul_award_big_prize.score == 5].info()

3362
<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 565 to 3341
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year           39 non-null     int64 
 1   film_festival  39 non-null     object
 2   title          39 non-null     object
 3   director       39 non-null     object
 4   field          39 non-null     object
 5   award          39 non-null     object
 6   genre          39 non-null     object
 7   plot           39 non-null     object
 8   rating         22 non-null     object
 9   runtime        38 non-null     Int64 
 10  prodYear       39 non-null     Int64 
 11  actors         35 non-null     object
 12  img_link       33 non-null     object
 13  stills         33 non-null     object
 14  score          39 non-null     Int64 
dtypes: Int64(3), int64(1), object(11)
memory usage: 5.0+ KB


#### 이외의 상은 10점 추가 (모든 영화제)

In [460]:
df_award_prize = df.copy()
df_award_prize.loc[(df_award_prize.award.notnull()),'score'] = 10
list_2 = np.array(df_award_prize['score'])
len(list_2)

3362

### 서울독립영화제에 전체 출품작 1073개 중에서 중복 출품한 감독들의 작품 개수는 481개

In [461]:
df_seoul_director = df.copy()
new_df = df.copy()
new_df = new_df[new_df.film_festival == '서울독립영화제']
new_df = new_df[new_df.duplicated(['director'], keep = False)]

for i,(ff_1, title_1, director_1) in enumerate(zip(df_seoul_director['film_festival'],df_seoul_director['title'],df_seoul_director['director'])):
    for ff_2, title_2, director_2 in zip(new_df['film_festival'],new_df['title'],new_df['director']):
        if (ff_1 == ff_2) & (title_1 == title_2) & (director_1 == director_2):
            df_seoul_director.loc[i,'score'] = 5
list_3 = np.array(df_seoul_director['score'])
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 481 entries, 3 to 3340
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year           481 non-null    int64 
 1   film_festival  481 non-null    object
 2   title          481 non-null    object
 3   director       481 non-null    object
 4   field          462 non-null    object
 5   award          55 non-null     object
 6   genre          481 non-null    object
 7   plot           458 non-null    object
 8   rating         141 non-null    object
 9   runtime        458 non-null    Int64 
 10  prodYear       463 non-null    Int64 
 11  actors         356 non-null    object
 12  img_link       418 non-null    object
 13  stills         383 non-null    object
 14  score          481 non-null    Int64 
dtypes: Int64(3), int64(1), object(11)
memory usage: 61.5+ KB


### 서울독립영화제에서 중복 수상한 감독은 3명으로 총 6개의 작품이 있다.

In [462]:
df_seoul_award_director = df.copy()
new_df = df.copy()
new_df = new_df[new_df.film_festival == '서울독립영화제']
new_df = new_df[new_df['award'].notnull()]
new_df = new_df[new_df.duplicated(['director'], keep = False)]

for i,(ff_1, title_1, director_1) in enumerate(zip(df_seoul_award_director['film_festival'],df_seoul_award_director['title'],df_seoul_award_director['director'])):
    for ff_2, title_2, director_2 in zip(new_df['film_festival'],new_df['title'],new_df['director']):
        if (ff_1 == ff_2) & (title_1 == title_2) & (director_1 == director_2):
            df_seoul_award_director.loc[i,'score'] = 5

list_4 = np.array(df_seoul_award_director['score'])
len(list_4)

3362

In [469]:
score = list_0 + list_1 +list_2 +list_3 +list_4
lis = np.sort(sc)[::-1]
df['score'] = score

In [470]:
df.to_csv('test.csv', encoding='utf8', index=False)